In [1]:
from numpy import array
from numpy import asarray
from numpy import zeros
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Reshape, Flatten, LSTM, Dense, Dropout, Embedding, Bidirectional, GRU,TimeDistributed
from keras.optimizers import Adam
from keras import initializers, regularizers, optimizers
from keras.utils.vis_utils import plot_model
import csv
import re
import sys

Using TensorFlow backend.


In [2]:
cd C:\Users\sachin\Desktop\fake news\LIAR PLUS dataset

C:\Users\sachin\Desktop\fake news\LIAR PLUS dataset


In [3]:
# import LIAR PLUS dataset
train=pd.read_csv('train2.tsv',delimiter='\t',encoding='utf-8')
test=pd.read_csv('test2.tsv',delimiter='\t',encoding='utf-8')


In [4]:
train.columns = ["index","ID" ,"label","statement","subject","speaker","speaker's job title","state info","party affiliation","barely true counts","false counts","half true counts","mostly true counts","pants on fire counts","the context","the extracted justification"]
train.columns = ["index","ID" ,"label","statement","subject","speaker","speaker's job title","state info","party affiliation","barely true counts","false counts","half true counts","mostly true counts","pants on fire counts","the context","the extracted justification"]


In [5]:
train

,index,ID,label,statement,subject,speaker,speaker's job title,state info,party affiliation,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,the context,the extracted justification
0,1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"Surovell said the decline of coal ""started whe..."
1,2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Obama said he would have voted against the ame...
2,3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,The release may have a point that Mikulskis co...
3,4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"Crist said that the economic ""turnaround start..."
4,5,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece,"But Vos specifically used the word ""fired,"" wh..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10234,10235,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,1.0,1.0,0.0,"interview on ""The Colbert Report""",They compounded their error by combining full ...
10235,10236,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,0.0,1.0,0.0,an interview,"Romney said that ""Obamacare means that for up..."
10236,10237,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,a Republican presidential debate,But that it leaves out important details and t...
10237,10238,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,3.0,0.0,0.0,a televised debate on Miami's WPLG-10 against ...,"We checked the research and, quite frankly, fi..."


In [6]:
train.shape

(10239, 16)

In [9]:
#drop nan values
count=0
for index, row in train.iterrows():
#     print(1)
    if type(train["the extracted justification"][index])!=str or type(train["statement"][index])!=str:
        count+=1
        train.drop(index, inplace=True)
        continue
#     x=len(train["the extracted justification"][index].split())
#     if x > 150:
#         train.drop(index, inplace=True)

In [10]:
count

84

In [11]:
train.shape

(10155, 16)

In [12]:
 # Preprocessing of the text, took care of basic pre-processing task only.

def processSent(sent):
    # To lowercase
    sent = sent.lower()
    #remove @username
    sent = re.sub(r'@(\w+)','',sent)
    
    # Remove hashtags
    sent = re.sub(r'#(\w+)', '', sent)
    
    # Remove Punctuation and split 's, 't, 've with a space for filter    
    sent = re.sub(r"[-()\"«/;»:<>{}`+=~|.!&?',]", "", sent)
    
    # remove the numerical values
    sent = re.sub(r"(\s\d+)","",sent) 
    
    # Remove HTML special entities (e.g. &amp;)
    sent = re.sub(r'\&\w*;', '', sent)
    
    # Remove tickers
    sent = re.sub(r'\$\w*', '', sent)
    
    # Remove hyperlinks
    sent = re.sub(r'https?:\/\/.*\/\w*', '', sent)
   
    # Remove whitespace (including new line characters)
    sent = re.sub(r'\s\s+', ' ', sent)
   
    # Remove single space remaining at the front of the sent.
    sent = sent.lstrip(' ') 
    
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    sent = ''.join(c for c in sent if c <= '\uFFFF') 
    
    return sent

In [13]:
data = train.copy()
data['the extracted justification'] = data['the extracted justification'].apply(processSent)
data['stattement'] = data['statement'].apply(processSent)
print(data["the extracted justification"].head())

0    surovell said the decline of coal started when...
1    obama said he would have voted against the ame...
2    the release may have a point that mikulskis co...
3    crist said that the economic turnaround starte...
4    but vos specifically used the word fired which...
Name: the extracted justification, dtype: object


In [14]:
train.label.value_counts()

half-true      2095
false          1975
mostly-true    1954
true           1664
barely-true    1637
pants-fire      830
Name: label, dtype: int64

In [15]:
docs1=data["the extracted justification"]
docs2=data["statement"]

In [16]:
# prepare tokenizer for "the extracted justification"
MAX_NB_WORDS = 50000
t1 = Tokenizer(num_words=MAX_NB_WORDS)
t1.fit_on_texts(docs1)
vocab_size = len(t1.word_index) + 1
# integer encode the documents
encoded_docs1 = t1.texts_to_sequences(docs1)
print(encoded_docs1)
# pad documents to a max length of 100 words
max_length = 100
padded_docs1 = pad_sequences(encoded_docs1, maxlen=max_length, padding='post')
print(padded_docs1)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
# prepare tokenizer "statement"
MAX_NB_WORDS = 50000
t2 = Tokenizer(num_words=MAX_NB_WORDS)
t2.fit_on_texts(docs2)
vocab_size = len(t2.word_index) + 1
# integer encode the documents
encoded_docs2 = t2.texts_to_sequences(docs2)
print(encoded_docs2)
# pad documents to a max length of 50 words
max_length = 50
padded_docs2 = pad_sequences(encoded_docs2, maxlen=max_length, padding='post')
print(padded_docs2)

[[63, 164, 1, 2073, 3, 958, 862, 23, 612, 63, 1131, 322, 181, 240, 9, 612, 4, 1950, 2, 34, 311, 566, 1423, 178], [126, 101, 3524, 24, 191, 253, 20, 330, 4, 340, 311, 166, 1, 1086, 3, 1, 3525, 11, 415], [31, 42, 263, 295, 10, 661, 4, 662, 402, 470, 416, 4130], [1, 337, 4131, 612, 38, 1, 408, 3, 176, 506], [1, 1817, 3526, 13, 82, 17, 1588, 5095, 2, 1, 89, 97, 33, 16, 1, 540, 219, 3, 5096, 2721, 2257, 1240, 201, 1, 89, 121, 959], [721, 6944, 12, 29, 2258, 2, 1, 370, 36, 2074, 8, 33, 85], [1305, 1, 68, 478, 11, 51, 1589, 60, 12, 722, 3075, 115, 89, 32, 1818, 1306, 24, 1006, 960, 187, 3, 1, 3076, 1132, 263, 79, 6945], [5097, 23, 181, 927, 140, 39, 2, 392, 2075, 471, 8, 1, 2472, 3, 3527, 4, 3528, 771, 1, 54, 5098, 3529, 1424, 1087, 1590], [7, 663, 642, 2722, 2259, 5099, 6, 613, 6946, 1357, 5, 2723, 169, 9, 151, 6947, 39, 2, 246, 1951, 256], [8, 1, 102, 95, 2, 192, 1, 1088, 3, 1, 137, 893, 169, 1952, 10, 1358, 16, 1, 2076, 169, 1952], [79, 772, 149, 271, 39, 109, 13, 6948, 69, 3, 1, 302, 332,

In [18]:
padded_docs1.shape

(10155, 100)

In [19]:
padded_docs2.shape

(10155, 50)

In [20]:
Y_train = pd.get_dummies(train['label']).values
print('Shape of label tensor:', Y_train.shape)

Shape of label tensor: (10155, 6)


In [21]:
Y_train

array([[0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       ...,
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0]], dtype=uint8)

In [22]:
cd C:\Users\sachin\Desktop\practice

C:\Users\sachin\Desktop\practice


In [23]:
# load the whole embedding into memory
import numpy as np
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,encoding="utf8")
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

In [24]:
embedding_glove =loadGloveModel("glove.42B.300d.txt")

Loading Glove Model
Done. 1917495  words loaded!


In [25]:
cd C:\Users\sachin\Desktop\fake news

C:\Users\sachin\Desktop\fake news


In [26]:
len(embedding_glove)

1917495

In [27]:
embedding_glove['men']

array([ 9.0092e-02,  8.1139e-02,  1.0442e-01, -3.7759e-01, -2.6276e-01,
        3.3072e-01, -2.7209e+00, -3.4311e-01, -3.2058e-01, -4.4907e-01,
        4.3773e-01, -2.6098e-01, -5.5974e-03,  8.6428e-02, -3.2471e-01,
        2.5996e-01, -7.9643e-03, -1.1521e-01,  5.8201e-01, -4.2476e-01,
       -2.4393e-02, -3.6907e-02, -6.7313e-01,  6.1443e-01,  2.7937e-01,
       -1.2075e-01, -2.6395e-01, -1.5605e-01, -1.5047e-02,  4.4229e-01,
       -7.8368e-01,  2.0017e-01, -3.2198e-01,  2.4271e-02, -2.9144e-01,
        3.6515e-01, -7.2606e-01,  6.3598e-01, -4.2325e-01, -5.1311e-01,
        3.0342e-01,  4.8570e-01, -1.8707e-01,  7.6418e-02, -6.0787e-02,
       -6.5920e-02, -6.7615e-02, -4.3819e-01,  4.4762e-01,  3.5316e-01,
        5.7285e-01, -1.2281e-01, -6.8277e-01, -3.2134e-01,  5.8772e-01,
        1.2765e-01,  2.2557e-01,  5.3913e-02, -6.7633e-01, -5.2345e-01,
       -2.1511e-02,  5.4085e-01,  5.7996e-01, -9.1423e-02,  1.7845e-01,
        2.9187e-01,  3.2739e-01, -2.2319e-01, -3.4587e-01, -2.39

In [28]:
# create a weight matrix for words in training docs1 "statement"
embedding_matrix1 = zeros((MAX_NB_WORDS, 300))
for word, i in t1.word_index.items():
	embedding_vector = embedding_glove.get(word)
	if embedding_vector is not None:
		embedding_matrix1[i] = embedding_vector

In [29]:
# create a weight matrix for words in training docs2 " the extracted justification"
embedding_matrix2 = zeros((MAX_NB_WORDS, 300))
for word, i in t2.word_index.items():
	embedding_vector = embedding_glove.get(word)
	if embedding_vector is not None:
		embedding_matrix2[i] = embedding_vector

In [35]:
embedding_matrix1

array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [-0.20838  , -0.14932  , -0.017528 , ..., -0.54066  ,  0.21199  ,
        -0.0094357],
       [-0.24837  , -0.45461  ,  0.039227 , ...,  0.053097 ,  0.15458  ,
        -0.38053  ],
       ...,
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]])

In [33]:
embedding_matrix1.shape[0]

50000

In [31]:
embedding_matrix2

array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [-0.20838  , -0.14932  , -0.017528 , ..., -0.54066  ,  0.21199  ,
        -0.0094357],
       [ 0.068507 , -0.023344 ,  0.28271  , ..., -0.23447  ,  0.4055   ,
        -0.28067  ],
       ...,
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]])

In [32]:
# from LSTM import lstm_word_embeddin

In [32]:
# importing from layers.py file
from layers import AttentionWithContext,Addition

In [33]:
#model for LSTM with variable number of layers, attention , bi_directional,pre_trained word_embedding.
def lstm_word_embedding(MAX_NB_WORDS, hidden_units, num_layers, max_sequence_length, is_attention, is_bidirectional, word_embedding):
	timesteps = max_sequence_length
	num_classes = 6
	embedding_dim = word_embedding.shape[1]
    
	adam = optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
	model = Sequential()
	model.add(Embedding(50000, 300, input_length=timesteps, weights=[word_embedding],trainable=False))
        #model.add(Embedding(MAX_NB_WORDS, embedding_dim, input_length=50, weights=[word_embedding],trainable=False))
	#model.add(Embedding(50000, 300, input_length=50, weights=[word_embedding],trainable=False))
       
	for i in range(num_layers):
		return_sequences = is_attention or (num_layers > 1 and i < num_layers-1)

		if is_bidirectional:
			model.add(Bidirectional(LSTM(hidden_units, return_sequences=return_sequences, dropout=0.2, kernel_initializer=initializers.glorot_normal(seed=777), bias_initializer='zeros')))
		else:
			model.add(LSTM(hidden_units, return_sequences=return_sequences, dropout=0.2, kernel_initializer=initializers.glorot_normal(seed=777), bias_initializer='zeros'))
		
		if is_attention and i==num_layers-1:
			model.add(AttentionWithContext())
			model.add(Addition())

	#model.add(Dense(num_classes, activation='softmax', kernel_initializer=initializers.glorot_normal(seed=777), bias_initializer='zeros'))
	#model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=["accuracy"])
	#model.summary()

In [34]:
#model for "the extracted justification"
model1=lstm_word_embedding(MAX_NB_WORDS,320,2,max_sequence_length=100,is_attention=True,is_bidirectional=False,word_embedding=embedding_matrix1)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [34]:
model1.add(Dense(6, activation='softmax', kernel_initializer=initializers.glorot_normal(seed=777), bias_initializer='zeros'))
model1.compile(loss='categorical_crossentropy',optimizer=adam,metrics=["accuracy"])
model1.summary()

NameError: name 'model1' is not defined

In [41]:
#model for "statement"
model2=lstm_word_embedding(MAX_NB_WORDS,320,2,max_sequence_length=50,is_attention=True,is_bidirectional=False,word_embedding=embedding_matrix2)

In [38]:
epochs=5
batch_size = 64
history = model.fit(padded_docs1, Y_train,validation_split=0.1,epochs=epochs,batch_size = batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 9139 samples, validate on 1016 samples
Epoch 1/5
9139/9139 [==============================] - 105s 11ms/step - loss: 1.7650 - acc: 0.2031 - val_loss: 1.7662 - val_acc: 0.1919
Epoch 2/5
9139/9139 [==============================] - 89s 10ms/step - loss: 1.7543 - acc: 0.2123 - val_loss: 1.7651 - val_acc: 0.1929
Epoch 3/5
9139/9139 [==============================] - 91s 10ms/step - loss: 1.7509 - acc: 0.2088 - val_loss: 1.7644 - val_acc: 0.2156
Epoch 4/5
9139/9139 [==============================] - 97s 11ms/step - loss: 1.7479 - acc: 0.2149 - val_loss: 1.7642 - val_acc: 0.2018
Epoch 5/5
9139/9139 [==============================] - 91s 10ms/step - loss: 1.7458 - acc: 0.2221 - val_loss: 1.7639 - val_acc: 0.2156


In [31]:
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, Y_train, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 25.782779


In [38]:
from keras.layers import Concatenate
from keras.models import Model
import keras

In [39]:
#model 1 for "the extracted justification"
model1_in = keras.layers.Input(shape=(100,))
# model1=Sequential()
model1_out = Embedding(MAX_NB_WORDS, 300, weights=[embedding_matrix1], input_length=100, trainable=False)(model1_in)

model1_out=LSTM(128,dropout_U=0.2,dropout_W=0.2,bias_initializer='zeros',return_sequences=True)(model1_out)
model1_out=LSTM(128,dropout_U=0.2,dropout_W=0.2,bias_initializer='zeros',return_sequences=True)(model1_out)
#model.add(Activation('softmax'))
model1_out=AttentionWithContext()(model1_out)
model1_out=Addition()(model1_out)
model1_out = keras.layers.Dense(6, activation='relu')(model1_out)

C:\Users\sachin\Anaconda3\envs\jupyter\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, bias_initializer="zeros", return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
  """


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Users\sachin\Anaconda3\envs\jupyter\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, bias_initializer="zeros", return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
  


In [40]:
#model for "statement"
model2_in = keras.layers.Input(shape=(50,))
# model1=Sequential()
model2_out = Embedding(MAX_NB_WORDS, 300, weights=[embedding_matrix2], input_length=50, trainable=False)(model2_in)

model2_out=LSTM(128,dropout_U=0.2,dropout_W=0.2,bias_initializer='zeros',return_sequences=True)(model2_out)
model2_out=LSTM(128,dropout_U=0.2,dropout_W=0.2,bias_initializer='zeros',return_sequences=True)(model2_out)
#model.add(Activation('softmax'))
model2_out=AttentionWithContext()(model2_out)
model2_out=Addition()(model2_out)
model2_out = keras.layers.Dense(6, activation='relu')(model2_out)
# model2_out.summary()

C:\Users\sachin\Anaconda3\envs\jupyter\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, bias_initializer="zeros", return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
  """
C:\Users\sachin\Anaconda3\envs\jupyter\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, bias_initializer="zeros", return_sequences=True, dropout=0.2, recurrent_dropout=0.2)`
  


In [41]:
merged=Concatenate()([model1_out,model2_out])
# merged
# concatenated = concatenate([model1_out, model2_out])
out = Dense(6, activation='softmax', name='output_layer', kernel_initializer=initializers.glorot_normal(seed=777), bias_initializer='zeros')(merged)

merged_model = Model([model1_in, model2_in], out)

In [42]:
merged_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     15000000    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 50, 300)      15000000    input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [43]:
adam = optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
# model.add(Dense(6, activation='softmax', kernel_initializer=initializers.glorot_normal(seed=777), bias_initializer='zeros'))
merged_model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=["accuracy"])
epochs=5
batch_size = 64
history = merged_model.fit([padded_docs1,padded_docs2], Y_train,validation_split=0.1,epochs=epochs,batch_size = batch_size)
# merged_model.summary()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 9139 samples, validate on 1016 samples
Epoch 1/5
9139/9139 [==============================] - 108s 12ms/step - loss: 1.7573 - acc: 0.2014 - val_loss: 1.7478 - val_acc: 0.2057
Epoch 2/5
9139/9139 [==============================] - 90s 10ms/step - loss: 1.7429 - acc: 0.2063 - val_loss: 1.7418 - val_acc: 0.2037
Epoch 3/5
9139/9139 [==============================] - 99s 11ms/step - loss: 1.7388 - acc: 0.2069 - val_loss: 1.7392 - val_acc: 0.2067
Epoch 4/5
9139/9139 [==============================] - 112s 12ms/step - loss: 1.7360 - acc: 0.2069 - val_loss: 1.7361 - val_acc: 0.2096
Epoch 5/5
9139/9139 [==============================] - 109s 12ms/step - loss: 1.7338 - acc: 0.2110 - val_loss: 1.7347 - val_acc: 0.2116
